이번 글에서는 [벡터 연산](http://wikibootup.github.io/sicp/2-2-vector-operations.html)에 이어서 고차함수를(`map`과 같은) 이용하여 두가지 알고리즘을 구현하겠습니다.

### 호너의 규칙

$a_nx^n + a_{n-1}x^{n-1} + ... + a_1x + a_0$

위의 다항식은 호너의 규칙에 따라 아래와 같이 바꿀 수 있습니다.

$(...(a_nx + a_{n-1})x + ... + a_1)x + a_0$

(특이한 공식을 사용한 것은 아니고 공통 인수인 $x$로 묶어둔 것입니다.) 

이것을 함수적으로 바꿔보면,

```HornerEval x higherTerms
-> FirstTerm + x * HornerEval x (higherTerms+1)
-> FirstTerm + x * (SecondTerm + x * HornerEval x (higherTerms+1))
-> ...
```

`map`과 `accumulate` 함수를 이용해 이런 생각을 그대로 옮겨보면,

In [113]:
hornerEval :: (Num a, Foldable t) => a -> t a -> a
hornerEval x coeffSeqs = foldl1 (\thisCoeff higherTerms -> x * thisCoeff + higherTerms) coeffSeqs

* 이전에 `Python`이나 `Scheme`(Git repo 상에 있는)에서는 `accumulate`라는 함수를 이용해서 리스트의 각 항에 필요한 연산을 적용했습니다. 여기서는 같은 기능인데 이름만 다른 `fold`라는 함수를 이용하였습니다.

In [114]:
x = 2

In [117]:
replicate 10 1

[1,1,1,1,1]

In [124]:
hornerEval 2 $ replicate 2 1

3

아래의 경우로 테스트해보겠습니다.

1. 계수 고정 : $2^{15} + 2^{14} + ... + 2 + 1 = 2^{16}-1$

2. 홀수 항만 : $2^{15} + 2^{13} + ... + 2^3 + 2^1 = ?$

3. 짝수 항만 : $2^{14} + 2^{12} + ... + 2^2 = ?$

4. 계수 변화 : $1*2^{15} + 2*2^{14} + ... + 16 = ?$


* 편의상 $x=2$로 고정하겠습니다.

In [285]:
hornerEval 2 (replicate 16 1) == 2^16-1

True

In [225]:
hornerEval 2 (concat (replicate 7 [1,0]) ++ [0]) == 
    foldl (\acc x -> 2^x + acc) 0 [x*2 | x <- [1..7]] 

True

In [226]:
hornerEval 2 (concat (replicate 8 [0,1]) ++ [0]) == 
    foldl (\acc x -> 2^x + acc) 0 [x*2-1 | x <- [1..8]] 

True

In [280]:
hornerEval 2 [1..16] == sum (zipWith (*) (map (\ x -> 2 ^ x) (reverse [0 .. 15])) [1..16])

True

### 퀸 퍼즐

In [54]:
flatmap :: (a1 -> [a]) -> [a1] -> [a]
flatmap f s = foldl1 (++) (map f s)

In [5]:
flatmap (\i -> map (\j -> [i,j]) [1..3]) [1..3]

[[1,1],[1,2],[1,3],[2,1],[2,2],[2,3],[3,1],[3,2],[3,3]]

In [6]:
:t flatmap

flatmap :: forall a1 a. (a1 -> [a]) -> [a1] -> [a]

8 Queens good reference
https://www.cs.usfca.edu/~galles/visualization/RecQueens.html

```
queen idea
1. 리스트에 위치 입력
맨 처음은 (0,0)에 퀸을 찍는다. 이 좌표를 리스트로 만들고 재귀 함수에 인자로 집어 넣는다. (이 때, 늦게 들어오는 퀸 위치를 리스트에 처음에 삽입한다)

2. 위치 탐색
그 다음은 (0,1)좌표에 퀸을 둘 수 있는지 인자로 넘어온 리스트를 통해 확인한다(직선과 대각선에서 만나지 않도록)

3. 백트래킹
(n,1)까지 그런 식으로 진행해서 가능한 위치가 존재하지 않는다면 이전에 지정한 리스트를 삭제하고 column은 -1 한 뒤 다시 시작한다.

4. 가능한 경우의 수 모두 수집


준비
1. (0 0)~(n-1 n-1)을 출력하는 함수를 만든다
2. setPosition
3. isSafe
```

In [130]:
isSafeVertical col colPosQueen = _isSafeVertical (0, col)
    where 
        _isSafeVertical (a,b)
            | a <= 7 = (a,b) /= colPosQueen && _isSafeVertical (a+1,b)
            | otherwise = True

In [95]:
isSafe tryPos posQueens = isSafeRow && isSafeCol && isSafeCross
    where
        isSafeRow = snd tryPos `notElem` map snd posQueens
        isSafeCol = fst tryPos `notElem` map fst posQueens
        isSafeCross = crossStartingPos tryPos `notElem` map crossStartingPos posQueens
            where
                crossStartingPos posQueens
                    | uncurry (<=) posQueens = (0, snd posQueens - fst posQueens)
                    | otherwise = (uncurry (-) posQueens, 0)

In [97]:
isSafe (0,0) p

True